# Automatic differentiation



**! Caution with the functions np.sort, np.where, np.stack, np.broadcast_to (silently fail) !**
Use replacements from the spad library, which also apply to np.ndarray.

In [2]:
import sys; sys.path.append("..") # Allow import from parent directory
from NumericalSchemes import SparseAutomaticDifferentiation as spad

In [3]:
import numpy as np

In [282]:
import importlib
spad = importlib.reload(spad)

arr = np.arange(5)
ad = (5-arr)+spad.identity(arr.shape)

In [274]:
ad/=(1+arr)


In [275]:
ad

(spAD(array([5. , 2. , 1. , 0.5, 0.2]), array([[1.        ],
        [0.5       ],
        [0.33333333],
        [0.25      ],
        [0.2       ]]), array([[0],
        [1],
        [2],
        [3],
        [4]])),)

In [284]:
sys.getsizeof(arr)

136

In [202]:
np.add(ad,arr)

adding spAD (array([5., 4., 3., 2., 1.]), array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]]), array([[0],
       [1],
       [2],
       [3],
       [4]])) and (spAD(array([5., 4., 3., 2., 1.]), array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]]), array([[0],
       [1],
       [2],
       [3],
       [4]])), array([0, 1, 2, 3, 4]))


In [208]:
ad[True]

spAD(array([[5., 4., 3., 2., 1.]]), array([[[1.],
        [1.],
        [1.],
        [1.],
        [1.]]]), array([[[0],
        [1],
        [2],
        [3],
        [4]]]))

# Sparse automatic differentiation

The sparse automatic differentiation class inherits from np.ndarray.

In [26]:
#arr = np.ones((2,3))
arr = np.arange(5)
ad = arr+spad.identity(arr.shape)

Elementary properties are inherited

In [6]:
ad.ndim,ad.shape,ad.size,len(ad)

(1, (5,), 5, 5)

However, internal data includes coefficient and index arrays, with one additional dimension.

In [7]:
ad.size_ad

1

Left and right multiplication, addition, substraction, divition, work as usual.

In [8]:
arr + 2*ad

spAD(array([ 0.,  3.,  6.,  9., 12.]), array([[2.],
       [2.],
       [2.],
       [2.],
       [2.]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

A number of elementary mathematical functions are implemented.

In [9]:
np.sqrt(1+ad)

spAD(array([1.        , 1.41421356, 1.73205081, 2.        , 2.23606798]), array([[0.5       ],
       [0.35355339],
       [0.28867513],
       [0.25      ],
       [0.2236068 ]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

In [286]:
np.abs(ad-2)

spAD(array([3., 2., 1., 0., 1.]), array([[ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [-1.]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

Comparison operators return an ndarray, as well as integer valued functions.

In [69]:
ad <= (-1+arr)

array([False, False, False,  True,  True])

In [70]:
np.floor(ad+0.5)

array([5., 4., 3., 2., 1.])

A base class ndarray can be recovered by requesting a view, or field 'value', or casting to np.array.

In [73]:
ad.view(np.ndarray), ad.value, np.array(ad)

(array([5., 4., 3., 2., 1.]),
 array([5., 4., 3., 2., 1.]),
 array([5., 4., 3., 2., 1.]))

maximum and minimum work as well

In [74]:
np.maximum(arr,ad)

spAD(array([5., 4., 3., 3., 4.]), array([[1.],
       [1.],
       [1.],
       [0.],
       [0.]]), array([[0],
       [1],
       [2],
       [0],
       [0]]))

### Caveats

**! Caution with the np.sort function (does nothing) !**
Calling it on a spAD type does nothing. Use the spad.sort function instead.

In [163]:
spad.sort(ad) # Correct
# np.sort(ad) # Viciously, does nothing

spAD(array([1., 2., 3., 4., 5.]), array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]]), array([[4],
       [3],
       [2],
       [1],
       [0]]))

**! Caution with np.where, np.stack, np.broadcast_to (silently cast to base class) !**
Some other functions numpy functions will cast their arguments to the base class.
, are not numpy universal functions, and their variant from the spad library needs to be called.

In [162]:
spad.where(ad<arr,ad,arr) # Correct
#np.where(ad<arr,ad,arr) # casts to ndarray

spAD(array([0., 1., 2., 2., 1.]), array([[0.],
       [0.],
       [0.],
       [1.],
       [1.]]), array([[0],
       [0],
       [0],
       [3],
       [4]]))

In [161]:
a,b = spad.identity((2,2))
spad.stack((a,b))
#np.stack((a,b)) # casts to ndarray

spAD(array([[0., 0.],
       [0., 0.]]), array([[[1.],
        [1.]],

       [[1.],
        [1.]]]), array([[[0],
        [1]],

       [[2],
        [3]]]))

**Non-silent failures:** np.reshape.

### Tested and working
np.take_along_axis, max, min, sum

In [185]:
ad.max(axis=0)

spAD(array(5.), array([1.]), array([0]))

In [191]:
np.min(ad,axis=0)

spAD(array(1.), array([1.]), array([4]))

In [196]:
np.sum(ad,axis=0)

spAD(array(15.), array([1., 1., 1., 1., 1.]), array([0, 1, 2, 3, 4]))